# 1.7.02 Pose Estimation

<h3>
    <ul>
        <li>We will learn to exploit calib3d module to create some 3D effects in images.</li>
    </ul>
    </h3>
    

In [6]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d point in image space

images = glob.glob('./board2/*.png')

print(len(images))

for fname in images :
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)
    
    if ret == True :
        objpoints.append(objp)
        
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv2.imshow('img', img)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        #print (mtx)
        #print (dist)
        #print (rvecs)
        cv2.waitKey(500)
    
cv2.destroyAllWindows()

5


In [7]:
import cv2
import numpy as np
import glob

# Load previously saved data

def drawAxis(img, corners, imgpts) :
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corners, tuple(imgpts[0].ravel()), (255, 0, 0), 5)
    img = cv2.line(img, corners, tuple(imgpts[1].ravel()), (0, 255, 0), 5)
    img = cv2.line(img, corners, tuple(imgpts[2].ravel()), (0, 0, 255), 5)
    return img

def drawBox(img, corners, imgpts) :
    imgpts = np.int32(imgpts).reshape(-1, 2)
    
    #draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), -3)
    
    #draw pillars in blue color
    for i, j in zip(range(4), range(4, 8)) :
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255), 3)
        
    #draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 3)
    
    return img
    
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

#axis = np.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]).reshape(-1, 3)
axis = np.float32([[0, 0, 0], [0, 3, 0], [3, 3, 0], [3, 0, 0],
                   [0, 0, -3], [0, 3, -3], [3, 3, -3], [3, 0, -3]])

for fname in glob.glob('./board2/*.png') :
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)
    cv2.imshow('img', img)
    
    if ret == True :
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        #Find the rotation and translation vectors.
        _, rvecs, tvecs, linliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)
        
        #Project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
        
        img = drawBox(img, corners2, imgpts)
        cv2.imshow('img', img)
        k = cv2.waitKey(0) & 0xff
    
cv2.destroyAllWindows()
    